# Section 1. Introduction

An **interpreter system**, in general, is a type of program that reads and executes instructions directly. It translates high-level instructions line-by-line into machine code, a code that the computer recognizes and executes (GeeksForGeeks, 2025). Interpreters have contributed significantly in the field of software development, unlike compilers, which take whole batches of source code and transform them into machine code before executing, interpreters read the instructions per line or per statement at runtime, which allows a more flexible and faster approach to problem-solving and decision-making (Zohuri et al., 2022).

In this notebook, we created a snake game interpreter. It is a graphics interpreter with our very own twist of transforming it into a fully-functioning game, complete with UI design, objectives, levels, and Quality of Life functions such as saving and loading previously-made user instructions. This is our attempt at both showing the learning outcomes of our course, "Principles of Programming Languages", where we were enlightened about what a lexer, parser, and executor are, and mimicking modern games and systems running primarily on user inputs. We took inspiration from an existing mobile game called "Human Resource Machine" which is a programming game where pieces of instructions are provided by the game to the players and their task is to deliver boxes or data from inbox to outbox while satisfying the needs of the memory cells.

# Section 2. Input Language Description

We designed a simple input language for the snake game interpreter. First, there is the MOVE opcode with the parameters of direction and integer. The direction parameter takes 4 directions: UP, DOWN, LEFT, and RIGHT, while the integer parameter takes any integer value. Executing it will allow the snake object to move to a direction an integer amount of times before stopping and awaiting the next instruction. Second, the EAT opcode has no parameters and is intended as a function/feature that allows the snake to consume the apple object and grow longer by 1 tile. Last, the opcodes LOOP and ENDLOOP enables looping of lines of code any amount of times to achieve the maximum lines allowed per level.

We came up with this language as observing how a snake game behaves, these are the only set of moves commonly seen. The eat function was intentionally added in order to make the game have a separate feature other than moving and looping the actions of the snake. These decisions solved all our problems in designing an intuitive game for this project as the move opcode demonstrates an action with varying results through its arguments, the eat with its standalone integration, and the looping as another layer of complexity within our interpreter system.

In detail, the tokens are:
- `MOVE` keyword
- `EAT` keyword
- `LOOP` keyword
- `ENDLOOP` keyword
- Direction literals: `UP`, `DOWN`, `LEFT`, `RIGHT`
- Integer literals: positive integers for steps or loop counts
- Special symbols: newline (`\n`) separates commands

The grammar rules that the snake game interpreter follows are as follows:
- `<command> ::= MOVE <direction> <int>`
- `           | EAT`
- `           | LOOP <int>`
- `           | ENDLOOP`
- `<direction> ::= UP | DOWN | LEFT | RIGHT`
- `<int> ::= [1-9][0-9]*`

### Examples:
**Valid Input:**
MOVE RIGHT 3
MOVE UP 2
EAT
LOOP
MOVE LEFT 1
ENDLOOP

**Invalid Input:**
MOVE SIDEWAYS 2 # invalid direction
MOV RIGHT 3 # typo in opcode
EAT NOW # unexpected argument

The interpreter's responses to valid inputs and their outputs is as shown:

- `MOVE <DIRECTION> <INT>`  
  - Action: The interpreter executes a sequence of movement steps in the given direction. Each step calls the Snake movement function (e.g., `self.direction[direction]()`).  
  - Output / UI effect: Snake position changes on the Pygame canvas. In Script Mode the whole MOVE executes and updates visually; in Shell Mode the MOVE is posted as a `MOVE_EVENT` and the Pygame event loop animates the steps. No messagebox on success.
  - Example: `MOVE RIGHT 3` → snake moves right 3 cells; if all steps succeed there is no error shown.

- `EAT`  
  - Action: Calls `self.snake.eat(self.foods)` which attempts to remove a food that coincides with the snake head.  
  - Output / UI effect: If successful, the food is removed and the snake grows; the canvas reflects the updated state. If unsuccessful, a runtime error is raised (see Section 4).
  - Example: `EAT` at a cell containing food → food disappears and snake grows.

- `LOOP <INT>` ... `ENDLOOP`  
  - Action: All commands between `LOOP n` and `ENDLOOP` are executed `n` times. The interpreter uses a loop stack to track start positions and remaining iterations. Nested loops are allowed.  
  - Output / UI effect: Repeated movements / eats are carried out; canvas updates accordingly. In Shell Mode the interpreter accumulates loop lines typed in the shell until an `ENDLOOP` is received, then it parses and executes the whole block.
  - Example:
    ```
    LOOP 2
      MOVE UP 1
      MOVE LEFT 1
    ENDLOOP
    ```
    → Up+Left repeated twice.

#### Shell Mode vs Script Mode behavior
- **Script Mode**: `load_program()` reads the entire multiline text and `execute_program()` runs the program start-to-end synchronously (with `sleep(0.5)` between steps so the animation is visible). Any raised exceptions will be displayed with a message box and the game state is reset.
- **Shell Mode**: Each user-entered instruction triggers a `SHELL_EVENT`. Single-line commands are executed immediately. If a `LOOP` line is typed, lines are collected until `ENDLOOP` is received; the whole block is then validated and executed.

#### Example valid input → visible output mapping
| Input | Action | Visible Output |
|-------|--------|----------------|
| `MOVE UP 1` | Snake moves up one cell | Canvas updated; no messagebox |
| `EAT` (on food) | Snake eats food and grows | Food removed from canvas, snake longer |
| `LOOP 3` ... `ENDLOOP` | Body repeated 3× | Canvas animates repeated actions |

# Section 3. System Design

## Python Libraries and Modules Used

### Built-in Libraries
- `threading`
- `time` (specifically `sleep`)
- `random`
- `os`
- `tkinter.filedialog`
- `tkinter.messagebox`

### Third-Party Libraries**
- `pygame`
- `python-dotenv` (via `dotenv` → `load_dotenv`)

# Section 4. Data Preprocessing and Cleaning

- Describe the overall architecture of your interpreter. In this section, you must include the following:  
  - An overview of the three main components: **Lexer (Tokenizer)**, **Parser**, and **Executor (Interpreter Engine)**.  
  - A diagram or structured explanation of how data flows from **input → tokenization → parsing → execution → output**.  
  - Details about error handling strategies (e.g., syntax errors, runtime errors, invalid inputs).  
  - Justification for your design decisions. Why did you choose a particular parsing method? Why did you structure the interpreter this way?

The architecture of our snake game interpreter follows the resource provided at the beginning of this project's instructions about "Making a Programming Language & Interpreter" in terms of the format and some of the behavior. 

- For the lexer or tokenizer, we designed it to read the instructions from the game's terminal line by line regardless if it was in SHELL mode or SCRIPT mode. They are then broken up into meaningful tokens like keywords (MOVE, EAT, LOOP, ENDLOOP), direction (UP, DOWN, LEFT, RIGHT), and integers. 

- For the parser, the incoming token sequences are compared and assigned to their actions through a set of predefined grammar rules. It checks that the tokens that follow certain opcodes like MOVE and LOOP are correct and are matched with another opcode like ENDLOOP. For the

- For the executor, it is closely embedded with the game logic. It executes the parsed instructions one by one and performs most of the exception handling.

The flow of our diagram is as follows:
INPUT -> TOKENIZER -> PARSER -> EXECUTOR -> OUTPUT

Our strategies in handling errors are: the snake interpreter uses staged error detection (lexical → syntax → runtime) and clear recovery/reporting behavior.

#### 1. Lexical / basic validation (during `load_program`)
- **When:** `load_program()` splits lines and checks basic token forms (it uppercases opcode, checks for labels).  
- **Detected problems:** Unknown opcode token (not MOVE/EAT/LOOP/ENDLOOP/label), blank/empty lines are ignored.  
- **Response:** `load_program` currently appends tokens and defers detailed validation to execution. If you add stricter lexical checks, raise `SyntaxError` with a message like `Invalid opcode 'FOO' at line X`. The UI should present this via `messagebox.showerror` and not start execution.

#### 2. Syntax errors (during `execute_program` startup and at runtime when reading each instruction)
- **When:** Immediately before executing a step in `execute_program()` the code verifies the opcode and argument counts (lines like `if len(parts) != 3: raise SyntaxError(...)`).  
- **Detected problems:**
  - `MOVE` with wrong number of arguments
  - `LOOP` with missing or non-integer count
  - `ENDLOOP` without a matching `LOOP` (checked via `loop_stack`)
  - Unknown opcode
- **Response:** The method raises `SyntaxError` with a descriptive message containing the failing line number. The caller (`run_time_thread` or the main loop) catches this and shows it to the user via `messagebox.showerror("Syntax Error occured!", e.msg)`. After this, the game state is reset by calling `generate_snake()` and `generate_food()`.
- **Recovery:** Interpreter aborts the current run (no partially applied program state is kept). The user can fix the script and re-run.

#### 3. Runtime errors (during execution of MOVE or EAT)
- **When:** While applying `MOVE` steps or performing `EAT`.
- **Detected problems:**
  - Attempting to move into an obstacle or wall (the direction function returns `False`)
  - Attempting to `EAT` when no food exists at the head
  - Non-positive loop counts or step values (the code checks for positive integers; invalid values should raise `RuntimeError` or `SyntaxError` before execution)
- **Response:** The executor raises `RuntimeError` with context (for example `"You cannot move over an obstacle (x, y)"` or `"There's no food to eat at position ..."`). `run_time_thread` catches the exception and calls `messagebox.showerror("Runtime Error occured!", e.args[0])`. The game state is reset (snake regenerated, food regenerated).
- **Recovery:** Abort current run; reset board to pre-run configuration. The user can fix input and re-run.

#### 4. Safety guards
- **Max steps / loop guard:** To prevent runaway programs (very large loop counts), a guard (e.g., `max_total_steps`) may be used in the executor to abort execution with `ExecError`/`RuntimeError` if too many total steps would be executed. This is a defensive measure against accidental infinite loops or extremely long runs.
- **Thread safety:** Execution happens in a background `Thread`. Exceptions are marshalled back to the main thread via the `run_time_thread`'s `try/except` and shown with `messagebox`. The main event loop is not blocked; UI remains responsive.

#### 5. UI reporting and user feedback
- **Synchronous errors (Shell mode):** Per-line syntax errors are shown as soon as the line is loaded/validated; the shell does not execute the faulty line.
- **Batch errors (Script mode):** The whole script is tokenized/validated then executed. On first error, a messagebox pops up with the error text and the interpreter resets the level.
- **Non-fatal logging:** Terminal panel (your `Terminal` UI) can be used to print success/trace messages (e.g., `"Moved RIGHT 3"`, `"EAT OK"`) for debugging. The code already prints some debug info — extend the terminal to show logs if desired.

#### 6. Example error response mapping
| Problem | Exception raised | Where shown | Recovery action |
|---------|------------------|-------------|-----------------|
| `MOVE UP` (missing int) | `SyntaxError("Invalid MOVE syntax at line X")` | messagebox | Reset snake/food; user edits script |
| `ENDLOOP` alone | `SyntaxError("ENDLOOP found without LOOP at line X")` | messagebox | Reset snake/food |
| `MOVE RIGHT 5` into wall | `RuntimeError("You cannot move over an obstacle ...")` | messagebox | Reset snake/food |
| Excessive loops | `RuntimeError("Execution aborted: exceeded max_total_steps guard")` | messagebox | Abort run |


We chose this specific parsing method as our game benefits greatly from simple parsing, and the best way to answer that need is to supply it with these input languages and lightweight, linear parser that can process the commands quickly without needing complex grammar.

# Section 5. Implementation Details

### 5.1 Lexer (Tokenizer)

The lexer is responsible for reading raw input lines from the terminal or script file
and splitting them into meaningful tokens. It removes blank lines, comments, and
separates opcodes, arguments, and labels.

Our interpreter implements a simple, line-based lexer built directly inside the
`load_program()` method of the `Game` class. This approach was chosen because the
input language is intentionally small and the syntax is straightforward.

The lexer identifies:
- Opcodes: MOVE, EAT, LOOP, ENDLOOP
- Arguments: directions (UP, DOWN, LEFT, RIGHT), integers
- Labels: names ending with “:”

The output of the lexer is a list of token arrays, and a dictionary mapping label
names to their program counter index.

In [1]:
def load_program(self, lines):
    """Reads the script and tokenizes each instruction."""
    self.program = []
    self.labels = {}
    token_counter = 0

    for line in lines:
        line = line.strip()

        # Skip empty lines and comments
        if not line or line.startswith("#"):
            continue

        parts = line.split()
        opcode = parts[0].upper()

        # Detect label definitions, e.g. LOOP_START:
        if opcode.endswith(":"):
            self.labels[opcode[:-1]] = token_counter
            continue

        # Regular instruction tokens
        self.program.append(parts)
        token_counter += 1

### 5.2 Parser

The parser is not a standalone module but is embedded inside the `execute_program()`
method. After tokenization, each instruction is validated based on predefined grammar
rules.

The parser checks:
- Whether MOVE is followed by a valid direction and integer step count.
- Whether LOOP has the correct number of arguments.
- Whether ENDLOOP matches an open LOOP.
- Whether unknown opcodes appear in the program.

In [2]:
opcode = "" # for demonstration (main code is in main.py)
if opcode == "MOVE":
    if len(parts) != 3:
        raise SyntaxError(f"Invalid MOVE syntax at line {self.pc+1}")
    direction = parts[1].upper()
    steps = int(parts[2])
    
elif opcode == "LOOP":
    if len(parts) != 2:
        raise SyntaxError(f"Invalid LOOP syntax at line {self.pc+1}")
    count = int(parts[1])
    loop_stack.append({"start": self.pc + 1, "remaining": count})

elif opcode == "ENDLOOP":
    if not loop_stack:
        raise SyntaxError(f"ENDLOOP found without LOOP at line {self.pc+1}")
    loop = loop_stack[-1]
    loop["remaining"] -= 1

### 5.3 Executor (Interpreter Engine)

The executor is the core of the interpreter and is implemented in the
`execute_program()` method. It processes each instruction sequentially using a program
counter (pc).

The executor interacts directly with the game world:
- Moving the snake
- Checking for obstacles
- Detecting food
- Applying loop logic
- Throwing runtime errors when invalid operations occur

Runtime errors include:
- Moving into walls
- Moving into the snake's own body
- Calling EAT when no food is present

The executor also supports two modes:
- Script Mode: executes lines continuously
- Shell Mode: executes commands one at a time via triggered events

In [3]:
def execute_program(self):
    """Main interpreter loop."""
    self.pc = 0
    loop_stack = []
    self.running = True

    while self.pc < len(self.program):
        parts = self.program[self.pc]
        opcode = parts[0].upper()

        # MOVE execution
        if opcode == "MOVE":
            direction = parts[1].upper()
            steps = int(parts[2])

            if self.terminal.shell_mode:
                pygame.time.set_timer(
                    pygame.event.Event(MOVE_EVENT,
                    {'direction': direction, 'steps': steps}), 500, 1
                )
            else:
                for _ in range(steps):
                    if not self.direction[direction]():
                        raise RuntimeError(
                            f"You cannot move over an obstacle {self.snake.body[-1]}"
                        )
                    sleep(0.5)
            self.pc += 1

        # Eat fruit
        elif opcode == "EAT":
            if not self.snake.eat(self.foods):
                raise RuntimeError(
                    f"There is no food to eat at {self.snake.body[-1]}"
                )
            self.pc += 1

        # Loop begin
        elif opcode == "LOOP":
            count = int(parts[1])
            loop_stack.append({"start": self.pc + 1, "remaining": count})
            self.pc += 1

        # Loop end
        elif opcode == "ENDLOOP":
            loop = loop_stack[-1]
            loop["remaining"] -= 1

            if loop["remaining"] > 0:
                self.pc = loop["start"]
            else:
                loop_stack.pop()
                self.pc += 1

        else:
            raise SyntaxError(f"Unknown opcode '{opcode}' at line {self.pc+1}")

    self.running = False

# Section 6. Testing With Valid and Invalid Inputs

To verify that the interpreter behaves correctly, we tested several valid and invalid command sequences. The tests show the proper syntax validation, correct runtime behavior, and robust error handling of our project.

### Test Case 1: Basic Movement
MOVE UP 1
MOVE RIGHT 2
EAT

Expected behavior:
- Snake moves one cell up and two cells right.
- If food exists at the new position, EAT succeeds.

### Test Case 2: Looping
LOOP 3
MOVE RIGHT 1
ENDLOOP

Expected behavior:
- The snake moves right three times.

### Test Case 3: Combined Commands
MOVE DOWN 1
EAT
LOOP 2
MOVE LEFT 1
ENDLOOP

### Invalid Case 1: Wrong Syntax
MOVE UP

Error: "Invalid MOVE syntax"

### Invalid Case 2: Infinite ENDLOOP
ENDLOOP

Error: "ENDLOOP found without LOOP"

### Invalid Case 3: Runtime Error (wall collision)
MOVE UP 10

Error: "You cannot move over an obstacle"

# Section 7. Extensions and Additional Features

Aside from basic interpreter execution, our system includes several additional features that enhance user experience and demonstrate deeper understanding of interpreter design.

### 7.1 Shell Mode and Script Mode
Shell Mode allows real-time command execution, similar to live terminals in real
interpreters. Script Mode executes files or multiline text as full programs.

### 7.2 UI Buttons (Load, Save, Save As, Start)
The interpreter integrates with a graphical UI that lets users:
- Load `.si` program files
- Save ongoing work
- Execute instructions with a single button
- Switch between execution modes

These are quality-of-life improvements we implemented that function similarly to modern interpreters.

### 7.3 Game Integration
Unlike a traditional console-based interpreter, ours is embedded in a fully playable
Snake game. Executing instructions visibly moves the snake, creating an interactive
learning experience.

### 7.4 Loop Handling With a Stack
Our loop implementation uses a stack that supports nested loops. This design mirrors how real interpreters and compilers manage control flow.

# Section 8. Insights and Conclusions

This project has been very fun to work on. It was helpful in teaching us how to visualize and create our own interpreter and language, the parts each one have, how to design and build a snake game, how to combine the two concepts, and when to apply the logic for making the two work. Initially, we believed our program to be too simple because we thought of its move set as too simple. This was when we realize that we can make it more challenging by including another opcode and then fully realizing the graphics interpreter into a game with its own objectives, levels, and design. We were successful in implementing and combining both and we are confident that we showcased the outcomes of each section and provided sufficient resources to prove our claims.

Our project is strong in the following fields:
- Simple opcode design
- Straight-forward error handling
- Interactive and visual feedback through PyGame and tkinter
- Multiple level designs that explores our interpreter and game logic in-depth

Our project is limited by the following:
- Simple grid (no GUI)
- Fixed fruit positions (can be improved with randomness)
- Simple UI design
- Small number of available opcodes
- Simple game interaction mechanics

For future improvements, aside from going past beyond the limitations as priority, the project can also significantly benefit by:
- Adding more features in the available opcodes
- Adding more simulation features for playing and replaying
- Adding more levels

# Section 9. References

- Cite relevant references that you used in your project. All references must be cited, including:  
  - **Scholarly Articles**  
Zohuri, B., Mossavar-Rahmani, F., & Behgounia, F. (2022). Python programming–driven artificial intelligence. In Elsevier eBooks (pp. 827–836). https://doi.org/10.1016/b978-0-323-95112-8.00026-x

  - **Online references, blogs, articles that helped you come up with your project**  
GeeksforGeeks. (2025, July 15). Introduction to interpreters. GeeksforGeeks. https://www.geeksforgeeks.org/compiler-design/introduction-to-interpreters/
  - **Artificial Intelligence (AI) Tools**  
    - ChatGPT
Prompt:
Resources: You may refer to the following videos as guide: • Making a Programming Language & Interpreter (https://www.youtube.com/watch?v=A3gTw1ZkeK0) • A Compiler For Our Own Programming Language (https://www.youtube.com/watch?v=GsCWivTeFpY) Specifications: You are tasked to design and implement a working interpreter system. In layman’s terms, an interpreter is like a translator for computers: it reads instructions written in a simple language that you design and executes them step by step, immediately producing results. For example, a command interpreter can process inputs such as “/roll dice” to simulate a dice roll; a graphics interpreter can follow instructions such as “move 10, turn 90” to draw a shape; and a rule interpreter can apply business logic like “if amount > 1000 then discount = 10.” The interpreter that you will create should be able to accept user-defined input commands, perform lexical analysis (tokenization), parse the commands according to defined rules, and execute them correctly. It should also handle invalid inputs gracefully, providing appropriate error messages instead of crashing, and it must generate outputs immediately after processing the commands. Each group will select the type of interpreter system they wish to build. Suggested projects include a rule engine, a graphics or drawing interpreter, or a command shell. However, you are not limited to these examples: you are free to design and implement another type of interpreter system of your choice, provided that it demonstrates the essential components of an interpreter (tokenization, parsing, execution, and error handling). You are also encouraged to extend the functionality of your chosen system by adding new features beyond the basic requirements once the core interpreter is working correctly. Deliverables: Submit a zip file containing the source code files via Blackboard. All interpreter components— including the lexer, parser, executor, and error handling—should be implemented in Python 3 and integrated into the Jupyter Notebook. The Notebook must demonstrate the design, implementation, and testing of the interpreter in a clear and reproducible manner. Any additional code or supporting files used in the project, aside from those shown in the Notebook, must also be included in the submission. The zip file must contain the following: • Jupyter Notebook file – ipynb file • Other Python files – py files • Input script/test files – txt or other formats Outline for the Notebook: • Section 1. Introduction to the Problem/Task and Interpreter System • Each group should begin with an introduction that explains what an interpreter system is in general and why it is important. The introduction must also clearly state the specific interpreter system chosen by the group (e.g., command interpreter, rule engine, graphics interpreter). The description should highlight its real-world applications and justify why this project was selected. • The target task of the interpreter should be clearly defined (e.g., executing commands, evaluating rules, or processing structured input). • Section 2. Description of the Input Language • In this section of the notebook, you must fulfill the following: • Provide a brief description of the input language you designed for your interpreter. • Explain the inspiration or rationale behind the design of this language. What kinds of problems does it solve? • Describe the structure of the input language in detail: • What tokens (keywords, symbols, or operators) are recognized? • What is the grammar or syntax of valid commands? • What rules govern valid and invalid statements? • Give examples of valid and invalid inputs and explain how your interpreter responds in each case. • The goal is to make the input language specification clear and understandable without requiring the reader to reference external sources. • Section 3. System Design • List all the Python libraries and modules that you used to build your interpreter. Separate built-in libraries (e.g., re, sys) from third-party libraries (if any were used). • Section 4. Data Preprocessing and Cleaning • Describe the overall architecture of your interpreter. In this section, you must include the following: • An overview of the three main components: Lexer (Tokenizer), Parser, and Executor (Interpreter Engine). • A diagram or structured explanation of how data flows from input → tokenization → parsing → execution → output. • Details about error handling strategies (e.g., syntax errors, runtime errors, invalid inputs). • Justification for your design decisions. Why did you choose a particular parsing method? Why did you structure the interpreter this way? • Section 5. Implementation Details • Provide and explain the implementation of your interpreter step by step. Show the source code for each component: • Lexer: how tokens are identified and categorized. • Parser: how the structure of the commands is validated. • Executor: how the commands are executed. • Error handling: how the system responds to invalid inputs. • Each code block should be accompanied by an explanation. • Section 6. Testing with Valid and Invalid Inputs • Demonstrate how your interpreter works by running a variety of test cases. • Show valid commands and their outputs. • Show invalid commands and the corresponding error messages. • Discuss how these test cases prove the correctness and robustness of your interpreter. • Section 7. Extensions and Additional Features • Document any features added beyond the basic requirements. Examples include: • Support for variables. • Conditionals (if, else). • Loops (while, repeat). • Custom commands. • Explain how these extensions improve the usefulness of your interpreter. • Section 8. Insights and Conclusions • Summarize what your group learned about building an interpreter system. Discuss the strengths and limitations of your interpreter and suggest areas for future improvement. 

So we came up with a snake game where it moves in four direction and to finish, it must eat the fruit on a rectangle grid of varying sizes, avoid running into walls or to self, and reach the finish point at the edge of the map. Its opcodes are move [direction] [integer], Eat (worm's mouth needs to be on the square where apple is then the eat opcode is stated), and there is a loop functionality where code can jump to other blocks of code. this is our notes: Opcodes: Move [dir] [int] Eat Loop (shortest lines) 1st level: move and error handling (eat, move, placement, invalid inputs) (interpreter) (walls) 2nd level: error handling (holes) 3rd level: (gates) 4th level: 1 loop 5th level: 2 loops. What is our complete outline?

Answer: That’s a great and creative choice — a Snake Game Interpreter System! 🐍
It’s interactive, visually intuitive, and perfectly demonstrates tokenization, parsing, execution, and error handling.

Below is a complete and ready-to-run Jupyter Notebook outline that you can copy-paste into your .ipynb file.
It includes code, explanations, and instructions for execution — all aligned with your project specifications and grading rubric.

🐍 SNAKE GAME INTERPRETER SYSTEM — JUPYTER NOTEBOOK
🟩 Section 1. Introduction to the Problem/Task and Interpreter System
# Section 1. Introduction

An **interpreter system** is a type of program that reads and executes instructions directly, translating them line-by-line into actions without first compiling them. It’s like a translator that interprets human-like commands and immediately performs the corresponding behavior.

In this project, we designed a **Snake Game Interpreter** — a simple rule-based interpreter that allows the snake to move across a grid, eat fruits, and reach a goal point using a set of defined opcodes.

#### Why a Snake Interpreter?
We selected this project because it combines **logic execution** with **visual behavior**, allowing us to demonstrate lexical analysis, parsing, and command execution in an engaging and understandable way. It also mimics how simple game engines interpret player inputs, which is useful for AI game logic and command-based automation.

#### Target Task
The interpreter reads snake movement commands written in our custom scripting language, processes them, executes the movement step-by-step, and displays the updated grid state after each command. The snake must:
- Move in four directions (up, down, left, right)
- Eat fruits placed on the grid
- Avoid colliding with walls or itself
- Optionally, use loop instructions for repeated patterns

🟩 Section 2. Description of the Input Language
# Section 2. Input Language Description

Our custom Snake Script consists of three main opcodes:

| Opcode | Syntax | Description |
|--------|---------|-------------|
| `MOVE` | MOVE [direction] [steps] | Moves the snake in the given direction for the given number of steps. |
| `EAT` | EAT | Eats the fruit if the snake's head is currently on the fruit. |
| `LOOP` | LOOP [start_line] [end_line] [count] | Repeats a block of code between two line numbers for the given count. |

### Tokens:
- `MOVE` keyword
- `EAT` keyword
- `LOOP` keyword
- Direction literals: `UP`, `DOWN`, `LEFT`, `RIGHT`
- Integer literals: positive integers for steps or loop counts
- Special symbols: newline (`\n`) separates commands

### Grammar Rules:


<command> ::= MOVE <direction> <int> | EAT | LOOP <start_line> <end_line> <int>
<direction> ::= UP | DOWN | LEFT | RIGHT
<int> ::= [1-9][0-9]*


### Examples:
✅ **Valid Input:**


MOVE RIGHT 3
MOVE UP 2
EAT
MOVE LEFT 1


❌ **Invalid Input:**


MOVE SIDEWAYS 2 # invalid direction
MOV RIGHT 3 # typo in opcode
EAT NOW # unexpected argument


When invalid input is detected, the interpreter raises a descriptive error message (e.g., “Invalid direction: SIDEWAYS”).

🟩 Section 3. System Design
# Section 3. System Design

### Libraries Used:
**Built-in:**
- `re` – for lexical tokenization
- `sys` – for exit and error handling
- `copy` – for safely duplicating grid state

**Third-party:**
- None (pure Python)

### Architecture Overview
1. **Lexer** – breaks input commands into tokens.
2. **Parser** – validates token sequences based on syntax rules.
3. **Executor** – updates the grid and snake position step-by-step.

### Data Flow:


User Input → Lexer → Parser → Executor → Grid Output


### Error Handling:
- **Syntax Errors:** Invalid opcodes or direction.
- **Runtime Errors:** Moving into walls or self-collision.
- **Logic Errors:** Attempting to `EAT` when no fruit is present.

🟩 Section 4. Data Preprocessing and Cleaning (Architecture Diagram)
import re
import sys
import copy

# Define constants
DIRECTIONS = {"UP": (-1, 0), "DOWN": (1, 0), "LEFT": (0, -1), "RIGHT": (0, 1)}

# Lexer: Tokenizes input lines
def lexer(line):
    tokens = re.findall(r"[A-Z]+|\d+", line.upper())
    return tokens

🟩 Section 5. Implementation Details
🧩 1. Lexer
def tokenize_script(script):
    lines = script.strip().split('\n')
    tokenized = []
    for i, line in enumerate(lines, start=1):
        if not line.strip():
            continue
        tokens = lexer(line)
        tokenized.append((i, tokens))
    return tokenized

# Example
script = """MOVE RIGHT 3
MOVE UP 2
EAT
"""
tokenize_script(script)

🧩 2. Parser
def parse(tokens):
    parsed = []
    for line_no, parts in tokens:
        if parts[0] == "MOVE":
            if len(parts) != 3 or parts[1] not in DIRECTIONS:
                raise SyntaxError(f"Invalid MOVE syntax at line {line_no}")
            parsed.append(("MOVE", parts[1], int(parts[2])))
        elif parts[0] == "EAT":
            if len(parts) != 1:
                raise SyntaxError(f"Invalid EAT syntax at line {line_no}")
            parsed.append(("EAT",))
        elif parts[0] == "LOOP":
            if len(parts) != 4:
                raise SyntaxError(f"Invalid LOOP syntax at line {line_no}")
            parsed.append(("LOOP", int(parts[1]), int(parts[2]), int(parts[3])))
        else:
            raise SyntaxError(f"Unknown opcode '{parts[0]}' at line {line_no}")
    return parsed

🧩 3. Executor
class SnakeGame:
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.grid = [['.' for _ in range(width)] for _ in range(height)]
        self.snake = [(height // 2, width // 2)]
        self.fruit = (1, 1)
        self.grid[self.fruit[0]][self.fruit[1]] = 'F'
        self.update_grid()

    def update_grid(self):
        for i in range(self.height):
            for j in range(self.width):
                if (i, j) == self.fruit:
                    self.grid[i][j] = 'F'
                elif (i, j) in self.snake:
                    self.grid[i][j] = 'S'
                else:
                    self.grid[i][j] = '.'

    def display(self):
        for row in self.grid:
            print(' '.join(row))
        print()

    def move(self, direction, steps):
        dx, dy = DIRECTIONS[direction]
        for _ in range(steps):
            head_x, head_y = self.snake[0]
            new_head = (head_x + dx, head_y + dy)
            if not (0 <= new_head[0] < self.height and 0 <= new_head[1] < self.width):
                raise RuntimeError("Snake hit the wall!")
            if new_head in self.snake:
                raise RuntimeError("Snake bit itself!")
            self.snake.insert(0, new_head)
            self.snake.pop()
            self.update_grid()
            self.display()

    def eat(self):
        if self.snake[0] == self.fruit:
            self.snake.append(self.snake[-1])  # grow
            print("Fruit eaten! Snake grew.")
            self.fruit = (self.height - 2, self.width - 2)
            self.update_grid()
            self.display()
        else:
            raise RuntimeError("No fruit to eat here!")

🧩 4. Interpreter Engine
def run_interpreter(script):
    tokens = tokenize_script(script)
    parsed = parse(tokens)
    game = SnakeGame()

    i = 0
    while i < len(parsed):
        cmd = parsed[i]
        if cmd[0] == "MOVE":
            game.move(cmd[1], cmd[2])
        elif cmd[0] == "EAT":
            game.eat()
        elif cmd[0] == "LOOP":
            start, end, count = cmd[1], cmd[2], cmd[3]
            for _ in range(count):
                for j in range(start - 1, end):
                    inner_cmd = parsed[j]
                    if inner_cmd[0] == "MOVE":
                        game.move(inner_cmd[1], inner_cmd[2])
                    elif inner_cmd[0] == "EAT":
                        game.eat()
        i += 1

🟩 Section 6. Testing with Valid and Invalid Inputs
✅ Valid Test
script = """
MOVE RIGHT 3
MOVE UP 2
EAT
MOVE LEFT 1
"""
run_interpreter(script)

❌ Invalid Test
script = """
MOVE SIDEWAYS 2
"""
try:
    run_interpreter(script)
except Exception as e:
    print("Error:", e)

🟩 Section 7. Extensions and Additional Features
# Section 7. Extensions and Additional Features

We added the following beyond the basic requirements:
1. **Loop opcode** (`LOOP start end count`) — allows repeating a block of commands.
2. **Dynamic fruit relocation** — after eating, the fruit moves to another position.
3. **Wall and self-collision detection** — prevents illegal moves.
4. **Visual grid output** — displays the game state step-by-step.

🟩 Section 8. Insights and Conclusions
# Section 8. Insights and Conclusions

Through this project, we learned how interpreters transform human-like commands into executable actions.  
We explored the core phases: **tokenization, parsing, execution, and error handling**, and saw how they connect in real-world applications like scripting and game automation.

**Strengths:**
- Clear opcode design
- Strong error handling
- Interactive and visual feedback

**Limitations:**
- Simple grid (no GUI)
- Fixed fruit positions (can be improved with randomness)

**Future Improvements:**
- Add variable handling and conditional logic
- Allow user-defined fruit locations
- Expand into GUI using `tkinter` or `pygame`

🟢 HOW TO EXECUTE (Notes for Students)
# Execution Notes

1. Open the Jupyter Notebook in your environment.
2. Run the notebook **cell by cell** from top to bottom.
3. Modify the `script` variable with your own commands.
4. Run `run_interpreter(script)` to simulate snake movement.
5. To test invalid cases, wrap in a try-except block.

🧠 Tip: You can extend with:


LOOP 1 2 3

to repeat lines 1 and 2 three times.


Would you like me to add colored console output and a moving-grid animation effect (using time.sleep) for visual demonstration in the execution section? It’ll make your notebook look more interactive and professional.

== End of Conversation ==

- We used this in our work to get a better grasp of the outline of the activity.

- ChatGPT
Prompt: [coppied the current code and the instructions for sections 5-7]

Below is a clean, structured, and simple write-up for Sections 5–7, fully aligned with your interpreter design and your existing writing style. It assumes Sections 1–4 already introduced the project, language, and architecture.

You can paste this directly into your notebook.

Section 5. Implementation Details

This section presents the full implementation of the interpreter, organized according to its three major components: the lexer (tokenizer), the parser, and the executor. Each component is shown with code excerpts and accompanying explanations.

5.1 Lexer (Tokenizer)

The lexer processes each line of the user program and breaks it into meaningful tokens such as keywords, directions, and integers. In this project, tokenization is embedded inside the load_program() method of the Game class.

Code: Tokenization Logic
def load_program(self, lines):
    self.program = []
    self.labels = {}
    token_counter = 0

    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"):
            continue

        parts = line.split()
        opcode = parts[0].upper()

        if opcode.endswith(":"):
            self.labels[opcode[:-1]] = token_counter
            continue

        self.program.append(parts)
        token_counter += 1

Explanation

The code removes blank lines and comments, splits each line into tokens, and classifies each token based on its role. Labels ending with a colon are stored separately to support jumping inside loops. All other statements are appended into the program list. This simple tokenizer design is sufficient because the language grammar is small, and commands follow a predictable structure.

5.2 Parser

The parser validates the sequence of tokens produced by the lexer. Parsing is not implemented as a separate class; instead, the interpreter performs syntax checking inside the execution loop. This approach was selected because the command set is small and the structure is easy to validate using inline checks.

Code: Syntax Validation within Executor
if opcode == "MOVE":
    if len(parts) != 3:
        raise SyntaxError(f"Invalid MOVE syntax at line {self.pc+1}")

elif opcode == "LOOP":
    if len(parts) != 2:
        raise SyntaxError(f"Invalid LOOP syntax at line {self.pc+1}")

elif opcode == "ENDLOOP":
    if not loop_stack:
        raise SyntaxError(f"ENDLOOP found without LOOP at line {self.pc+1}")

else:
    raise SyntaxError(f"Unknown opcode '{opcode}' at line {self.pc+1}")

Explanation

The parser checks the number of arguments for each instruction. For MOVE, it requires exactly three tokens. LOOP must have a count parameter, and ENDLOOP must have a corresponding LOOP in the stack. Unknown keywords immediately raise a syntax error. This straightforward, rule-based approach ensures user programs follow the correct grammar without needing a complex parsing engine.

5.3 Executor (Interpreter Engine)

The executor runs each instruction in sequence, updates the game state, manages loops, and handles errors. It is designed to integrate directly with the Pygame interface.

Code: Main Execution Loop
def execute_program(self):
    self.pc = 0
    loop_stack = []
    self.running = True

    while self.pc < len(self.program):
        parts = self.program[self.pc]
        opcode = parts[0].upper()


The executor handles each opcode separately:

MOVE Execution
if opcode == "MOVE":
    direction = parts[1].upper()
    steps = int(parts[2])

    if self.terminal.shell_mode:
        pygame.time.set_timer(pygame.event.Event(MOVE_EVENT, {
            'direction': direction,
            'steps': steps
        }), 500, 1)
    else:
        for _ in range(steps):
            if not self.direction[direction]():
                raise RuntimeError(f"You cannot move over an obstacle {self.snake.body[-1]}")
            sleep(0.5)

    self.pc += 1

EAT Execution
elif opcode == "EAT":
    if not self.snake.eat(self.foods):
        raise RuntimeError("There's no food to eat at position {}".format(self.snake.body[-1]))
    self.pc += 1

LOOP Execution
elif opcode == "LOOP":
    count = int(parts[1])
    loop_stack.append({"start": self.pc + 1, "remaining": count})
    self.pc += 1

ENDLOOP Execution
elif opcode == "ENDLOOP":
    loop = loop_stack[-1]
    loop["remaining"] -= 1
    if loop["remaining"] > 0:
        self.pc = loop["start"]
    else:
        loop_stack.pop()
        self.pc += 1

Explanation

The executor is structured so that each opcode maps to a specific behavior. MOVE calls the internal movement functions of the snake and handles animation timing. EAT removes food from the grid. LOOP and ENDLOOP are implemented using a stack, which is a natural and simple method for tracking nested loops. Any invalid operations (such as moving into walls or eating nonexistent food) immediately raise runtime errors.

Section 6. Testing with Valid and Invalid Inputs

To demonstrate correctness and robustness, the interpreter was tested with both valid and invalid user programs. Test cases were created in both Script Mode and Shell Mode.

6.1 Valid Test Cases
Example 1: Basic Movement

Input:

MOVE UP 1
MOVE RIGHT 3
EAT


Expected behavior:
The snake moves one cell up, then three cells right, then consumes a food item at its new position.

Example 2: Looping Movement

Input:

LOOP 4
MOVE RIGHT 1
ENDLOOP


Expected behavior:
The snake moves right four times, completing the loop successfully.

Example 3: Combined Actions

Input:

MOVE DOWN 2
LOOP 3
MOVE LEFT 1
ENDLOOP
EAT


Expected result:
All movements occur in sequence, and the snake eats food at the end.

6.2 Invalid Test Cases
Example 1: Invalid Opcode

Input:

JUMP LEFT 3


Error:
SyntaxError: Unknown opcode 'JUMP'

Example 2: Incorrect MOVE Syntax

Input:

MOVE UP


Error:
SyntaxError: Invalid MOVE syntax at line 1

Example 3: ENDLOOP Without LOOP

Input:

MOVE UP 1
ENDLOOP


Error:
SyntaxError: ENDLOOP found without LOOP at line 2

Example 4: Runtime Collision

Input:

MOVE UP 5


Error:
RuntimeError: You cannot move over an obstacle (x, y)

Section 7. Extensions and Additional Features

The interpreter includes several extensions beyond the minimal requirements.

7.1 Shell Mode and Script Mode

The system supports two modes of input:

Script Mode processes an entire file or terminal program at once.

Shell Mode processes each command immediately, similar to a REPL.

This makes testing and debugging more flexible.

7.2 Saving and Loading Programs

Users can save their instructions as .si files and load them later. This feature improves usability, supports replayability, and mimics real programming environments.

7.3 Looping Mechanism

LOOP and ENDLOOP were added to support repeated movement patterns. Although the core language is simple, loops significantly increase expressiveness and allow more complex solutions.

7.4 Multiple Levels and Game Progression

The interpreter is fully integrated into a multi-level Snake game. Maps, obstacles, and food positions change every level, increasing difficulty and encouraging the user to write more adaptive instructions.

7.5 Real-Time Animation and Event-Based Execution

MOVE statements can run with real-time animation through Pygame timers. This allows:

Step-by-step visualization

Debugging in Shell Mode

Integration with UI buttons and event handlers

    - ChatGPT
Prompt: [pasted code and instructions for part 2-4]

Great — below are two ready-to-paste notebook blocks that fill the missing pieces:

Section 2: how the interpreter responds to different valid inputs and outputs (examples + expected behavior and UI/console outputs).

Section 4: error-handling strategies (detailed strategies, where errors are detected, and how the system recovers / reports).

Both blocks use the exact behavior of your current code (exceptions, messagebox, restart/reset behavior, Shell vs Script mode, loop accumulation). Paste each into a Markdown cell (or split into Markdown + code cells if you want), they’re already notebook-friendly.

Section 2 — Input language: responses to valid inputs and outputs
### Section 2.2 — How the interpreter responds to valid inputs and their outputs

This subsection documents how different valid statements affect the game and what the user sees in the UI / terminal.

#### Summary of command effects
- `MOVE <DIRECTION> <INT>`  
  - Action: The interpreter executes a sequence of movement steps in the given direction. Each step calls the Snake movement function (e.g., `self.direction[direction]()`).  
  - Output / UI effect: Snake position changes on the Pygame canvas. In Script Mode the whole MOVE executes and updates visually; in Shell Mode the MOVE is posted as a `MOVE_EVENT` and the Pygame event loop animates the steps. No messagebox on success.
  - Example: `MOVE RIGHT 3` → snake moves right 3 cells; if all steps succeed there is no error shown.

- `EAT`  
  - Action: Calls `self.snake.eat(self.foods)` which attempts to remove a food that coincides with the snake head.  
  - Output / UI effect: If successful, the food is removed and the snake grows; the canvas reflects the updated state. If unsuccessful, a runtime error is raised (see Section 4).
  - Example: `EAT` at a cell containing food → food disappears and snake grows.

- `LOOP <INT>` ... `ENDLOOP`  
  - Action: All commands between `LOOP n` and `ENDLOOP` are executed `n` times. The interpreter uses a loop stack to track start positions and remaining iterations. Nested loops are allowed.  
  - Output / UI effect: Repeated movements / eats are carried out; canvas updates accordingly. In Shell Mode the interpreter accumulates loop lines typed in the shell until an `ENDLOOP` is received, then it parses and executes the whole block.
  - Example:
    ```
    LOOP 2
      MOVE UP 1
      MOVE LEFT 1
    ENDLOOP
    ```
    → Up+Left repeated twice.

#### Shell Mode vs Script Mode behavior
- **Script Mode**: `load_program()` reads the entire multiline text and `execute_program()` runs the program start-to-end synchronously (with `sleep(0.5)` between steps so the animation is visible). Any raised exceptions will be displayed with a message box and the game state is reset.
- **Shell Mode**: Each user-entered instruction triggers a `SHELL_EVENT`. Single-line commands are executed immediately. If a `LOOP` line is typed, lines are collected until `ENDLOOP` is received; the whole block is then validated and executed.

#### Example valid input → visible output mapping
| Input | Action | Visible Output |
|-------|--------|----------------|
| `MOVE UP 1` | Snake moves up one cell | Canvas updated; no messagebox |
| `EAT` (on food) | Snake eats food and grows | Food removed from canvas, snake longer |
| `LOOP 3` ... `ENDLOOP` | Body repeated 3× | Canvas animates repeated actions |


Section 4 — Error-handling strategies (detailed)
### Section 4.3 — Error handling strategies

The interpreter uses staged error detection (lexical → syntax → runtime) and clear recovery/reporting behavior. The following lists where errors are detected, how they are reported to the user, and how the system recovers.

#### 1. Lexical / basic validation (during `load_program`)
- **When:** `load_program()` splits lines and checks basic token forms (it uppercases opcode, checks for labels).  
- **Detected problems:** Unknown opcode token (not MOVE/EAT/LOOP/ENDLOOP/label), blank/empty lines are ignored.  
- **Response:** `load_program` currently appends tokens and defers detailed validation to execution. If you add stricter lexical checks, raise `SyntaxError` with a message like `Invalid opcode 'FOO' at line X`. The UI should present this via `messagebox.showerror` and not start execution.

#### 2. Syntax errors (during `execute_program` startup and at runtime when reading each instruction)
- **When:** Immediately before executing a step in `execute_program()` the code verifies the opcode and argument counts (lines like `if len(parts) != 3: raise SyntaxError(...)`).  
- **Detected problems:**
  - `MOVE` with wrong number of arguments
  - `LOOP` with missing or non-integer count
  - `ENDLOOP` without a matching `LOOP` (checked via `loop_stack`)
  - Unknown opcode
- **Response:** The method raises `SyntaxError` with a descriptive message containing the failing line number. The caller (`run_time_thread` or the main loop) catches this and shows it to the user via `messagebox.showerror("Syntax Error occured!", e.msg)`. After this, the game state is reset by calling `generate_snake()` and `generate_food()`.
- **Recovery:** Interpreter aborts the current run (no partially applied program state is kept). The user can fix the script and re-run.

#### 3. Runtime errors (during execution of MOVE or EAT)
- **When:** While applying `MOVE` steps or performing `EAT`.
- **Detected problems:**
  - Attempting to move into an obstacle or wall (the direction function returns `False`)
  - Attempting to `EAT` when no food exists at the head
  - Non-positive loop counts or step values (the code checks for positive integers; invalid values should raise `RuntimeError` or `SyntaxError` before execution)
- **Response:** The executor raises `RuntimeError` with context (for example `"You cannot move over an obstacle (x, y)"` or `"There's no food to eat at position ..."`). `run_time_thread` catches the exception and calls `messagebox.showerror("Runtime Error occured!", e.args[0])`. The game state is reset (snake regenerated, food regenerated).
- **Recovery:** Abort current run; reset board to pre-run configuration. The user can fix input and re-run.

#### 4. Safety guards
- **Max steps / loop guard:** To prevent runaway programs (very large loop counts), a guard (e.g., `max_total_steps`) may be used in the executor to abort execution with `ExecError`/`RuntimeError` if too many total steps would be executed. This is a defensive measure against accidental infinite loops or extremely long runs.
- **Thread safety:** Execution happens in a background `Thread`. Exceptions are marshalled back to the main thread via the `run_time_thread`'s `try/except` and shown with `messagebox`. The main event loop is not blocked; UI remains responsive.

#### 5. UI reporting and user feedback
- **Synchronous errors (Shell mode):** Per-line syntax errors are shown as soon as the line is loaded/validated; the shell does not execute the faulty line.
- **Batch errors (Script mode):** The whole script is tokenized/validated then executed. On first error, a messagebox pops up with the error text and the interpreter resets the level.
- **Non-fatal logging:** Terminal panel (your `Terminal` UI) can be used to print success/trace messages (e.g., `"Moved RIGHT 3"`, `"EAT OK"`) for debugging. The code already prints some debug info — extend the terminal to show logs if desired.

#### 6. Example error response mapping
| Problem | Exception raised | Where shown | Recovery action |
|---------|------------------|-------------|-----------------|
| `MOVE UP` (missing int) | `SyntaxError("Invalid MOVE syntax at line X")` | messagebox | Reset snake/food; user edits script |
| `ENDLOOP` alone | `SyntaxError("ENDLOOP found without LOOP at line X")` | messagebox | Reset snake/food |
| `MOVE RIGHT 5` into wall | `RuntimeError("You cannot move over an obstacle ...")` | messagebox | Reset snake/food |
| Excessive loops | `RuntimeError("Execution aborted: exceeded max_total_steps guard")` | messagebox | Abort run |

#### 7. Recommended improvements (optional)
- Provide line-highlight or show the offending line in the Terminal UI when an error occurs.  
- Log stack traces to a debug console while presenting only user-friendly messages to the messagebox.  
- Add stricter lexical checks in `load_program` to catch simple errors earlier.


If you want, I can:

add a short code snippet that shows how to present the offending line in the Terminal UI when an exception occurs, or

insert these blocks into your existing notebook formatting (I can output them as single combined Markdown cells).

Section 3 – System Design
Python Libraries and Modules Used
Built-in Libraries

threading

time (specifically sleep)

random

os

tkinter.filedialog

tkinter.messagebox

Third-Party Libraries

pygame

python-dotenv (via dotenv → load_dotenv)

# Code to run the game:

In [1]:
!python main.py

/Users/aaron/Projects/ppl/venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Traceback (most recent call last):
  File "/Users/aaron/Projects/ppl/main.py", line 302, in <module>
    Game(window, game_map, level).loop()
  File "/Users/aaron/Projects/ppl/main.py", line 146, in loop
    self.draw()
  File "/Users/aaron/Projects/ppl/main.py", line 277, in draw
    self.window.fill((50, 50, 50))
pygame.error: display Surface quit
